In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import tifffile
import sys, time, os, csv, warnings
import cProfile, pstats
from jutils import tqdm_alias as tqdm

import psfmatrix, lfimage
import projector, lfdeconv
import special_fftconvolve as special
import jutils as util
import py_light_field as plf

In [ ]:
projectorClass = projector.Projector_allC
inputImage = lfimage.LoadLightFieldTiff('/Users/jonny/Movies/Nils files/Rectified/Left/Cam_Left_40_X1_N19.tif')
hMatrix = psfmatrix.LoadMatrix('PSFmatrix/reducedPSFMatrix_M22.2NA0.5MLPitch125fml3125from-156to156zspacing4Nnum19lambda520n1.33.mat')

In [ ]:
if True:
    # Timing measurements
    tileFactor = 1

    inputImageXN = np.tile(inputImage[np.newaxis,:,:],(tileFactor,1,1))
    # We will only process one plane, in the interests of speed,
    # although we must remember that that means the analysis may not 
    # be representative of a full deconvolution pipeline
    planesToRun = range(0,1)
    # Prime the cache first
    Htf = lfdeconv.BackwardProjectACC(hMatrix, inputImageXN, planes=planesToRun, progress=tqdm, logPrint=False, projector=projectorClass())
    # Now actually measure
    plf.SetStatsFile('timestats.txt', False)
    Htf = lfdeconv.BackwardProjectACC(hMatrix, inputImageXN, planes=planesToRun, progress=tqdm, logPrint=False, projector=projectorClass())
    plf.SetStatsFile('', True)

In [ ]:
rows = []
with open('threads.txt') as f:
    cf = csv.reader(f, delimiter='\t')
    for row in cf:
        rows.append(np.array(row).astype(np.double))
t0 = rows[0][2]
col = ['red', 'green', 'blue']
lab = ['FFT', 'mirror', 'convolve']
plt.figure(figsize=(14,4))
reportWaits = 10
runTime = 0
mutexTime = 0
for w in range(3):
    for t in range(8):
        x = []; y = []
        mx = []; my = []
        intermediateTimes = []
        for r in rows:
            if (r[0] == t) and (r[1] == w):
                x.extend([r[2], r[2], r[3], r[3]])
                y.extend([0,1,1,0])
                runTime += r[3]-r[2]
                if (w == 2):
                    if ((r[5]-r[4]) > 5e-6):
                        if (reportWaits):
                            print("Mutex wait: %le"%(r[5]-r[4]))
                            reportWaits = reportWaits - 1
                        mx.extend([r[4], r[4], r[5], r[5]])
                        my.extend([0,1,1,0])
                    mutexTime += r[5]-r[4]
                    if (r[6] != 0) and ((r[7]-r[6]) > 1e-5):
                        if (reportWaits):
                            print("Mutex wait: %le"%(r[7]-r[6]))
                            reportWaits = reportWaits - 1
                        mx.extend([r[6], r[6], r[7], r[7]])
                        my.extend([0,1,1,0])
                    mutexTime += r[7]-r[6]
                if (w == 0):
                    intermediateTimes.extend([r[5], r[7]])

                        
        x = np.array(x) - t0
        y = np.array(y)
        mx = np.array(mx) - t0
        my = np.array(my)
        intermediateTimes = np.array(intermediateTimes) - t0
        if t == 0:
            thisLabel = lab[w]
        else:
            thisLabel = None
        # Show the time spent working on a task
        plt.fill_between(x, t, t+y/2, color=col[w], where=y.astype(np.bool), label=thisLabel)
        # Show the time spent waiting to acquire an accum mutex.
        plt.fill_between(mx, t+0.1, t+0.1+my*0.3, where=my.astype(np.bool), color='gray')
        if (w == 0):
            plt.plot(intermediateTimes, [t+0.25]*len(intermediateTimes), 'x', color='black')

print('Run %.2f, active %.2f, mutex %.2f, idle frac %.2f' % (runTime, runTime-mutexTime, mutexTime, mutexTime/runTime))
#        plt.xlim(0.1,1)
if (reportWaits == 0):
    warnings.warn('Did not report all waits')
plt.legend()
plt.show()            